# Test Notebook: Executing optimizer.py and capacity_optimizer.py (step-by-step)

This notebook runs the raw functions with detailed prints: recursive route generation, PuLP LP formulation output, and step-by-step data transformations.

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta
from pulp import *
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)
print('✓ Imports OK - Python', sys.version.split()[0])

✓ Imports OK - Python 3.10.0


In [2]:
VEHICLE_SPECS = {
    'truck': {'speed': 80, 'cost': 0.50},
    'van': {'speed': 90, 'cost': 0.40},
    'car': {'speed': 100, 'cost': 0.30},
    'auto': {'speed': 60, 'cost': 0.20},
    'bike': {'speed': 25, 'cost': 0.10},
    'plane': {'speed': 900, 'cost': 2.00},
}
print('✓ VEHICLE_SPECS loaded')

✓ VEHICLE_SPECS loaded


In [4]:
def enumerate_paths(nodes, edges, start, end, max_depth=10):
    """Recursively enumerate simple paths from start to end and print recursion steps.
    Returns list of paths (each path is list of nodes).
    """
    print(f'\n[enumerate_paths] Starting: start={start}, end={end}, max_depth={max_depth}')
    paths = []
    def dfs(current, path, depth):
        indent = "  " * depth
        print(f"{indent}[RECURSION depth={depth}] current={current}, path={path}")
        
        # Line: Check max depth
        if depth > max_depth:
            print(f'{indent}  → Max depth ({max_depth}) reached, backtracking')
            return
        
        # Line: Check if reached destination
        if current == end:
            print(f'{indent}  → FOUND PATH: {path}')
            paths.append(list(path))
            return
        
        # Line: Iterate through all edges
        for edge_idx, (u, v) in enumerate(edges.keys()):
            print(f'{indent}  [edge {edge_idx}] ({u}, {v}): u==current? {u==current}, v in path? {v in path}')
            
            if u == current and v not in path:
                print(f'{indent}    ✓ Valid edge found, adding {v} to path')
                path.append(v)
                print(f'{indent}    → Recursing with path={path}')
                dfs(v, path, depth+1)
                path.pop()
                print(f'{indent}    ← Backtracked from {v}')
            else:
                reason = "u != current" if u != current else "v in path"
                print(f'{indent}    ✗ Edge skipped ({reason})')
    
    dfs(start, [start], 0)
    print(f'\n[enumerate_paths] Completed. Total enumerated paths: {len(paths)}')
    for i, p in enumerate(paths, 1):
        print(f'  Path {i}: {" → ".join(str(n) for n in p)}')
    return paths

print('✓ enumerate_paths defined')

# ============================================================================
# TEST 1: enumerate_paths with dummy simple graph
# ============================================================================
print('\n' + '#'*70)
print('# TEST 1: enumerate_paths Function')
print('#'*70)

test_nodes_1 = {
    'A': (0, 0),
    'B': (1, 1),
    'C': (2, 0),
    'D': (3, 1)
}
test_edges_1 = {
    ('A', 'B'): {'distance': 10},
    ('B', 'C'): {'distance': 15},
    ('A', 'C'): {'distance': 20},
    ('C', 'D'): {'distance': 12},
    ('B', 'D'): {'distance': 25}
}

print('\nInput: Simple DAG with 4 nodes')
print(f'  Nodes: {list(test_nodes_1.keys())}')
print(f'  Edges: {list(test_edges_1.keys())}')
print(f'  Finding paths from A to D')

paths_result = enumerate_paths(test_nodes_1, test_edges_1, 'A', 'D', max_depth=5)
print(f'\n✓ Test 1 complete - Found {len(paths_result)} paths')


✓ enumerate_paths defined

######################################################################
# TEST 1: enumerate_paths Function
######################################################################

Input: Simple DAG with 4 nodes
  Nodes: ['A', 'B', 'C', 'D']
  Edges: [('A', 'B'), ('B', 'C'), ('A', 'C'), ('C', 'D'), ('B', 'D')]
  Finding paths from A to D

[enumerate_paths] Starting: start=A, end=D, max_depth=5
[RECURSION depth=0] current=A, path=['A']
  [edge 0] (A, B): u==current? True, v in path? False
    ✓ Valid edge found, adding B to path
    → Recursing with path=['A', 'B']
  [RECURSION depth=1] current=B, path=['A', 'B']
    [edge 0] (A, B): u==current? False, v in path? True
      ✗ Edge skipped (u != current)
    [edge 1] (B, C): u==current? True, v in path? False
      ✓ Valid edge found, adding C to path
      → Recursing with path=['A', 'B', 'C']
    [RECURSION depth=2] current=C, path=['A', 'B', 'C']
      [edge 0] (A, B): u==current? False, v in path? True
       

In [ ]:
def optimize(nodes, edges, start, end, objective='cost', via=None, lp_filename='model.lp', solver_msg=True):
    """Build LP problem, print full formulation, write LP file, enumerate paths, and solve with CBC.
    Also prints selection of edges and solver output.
    """
    print('\n' + '='*60)
    print('[OPTIMIZE] Input Analysis')
    print('='*60)
    print(f'  start: {start}, end: {end}')
    print(f'  objective: {objective}, via: {via}')
    print(f'  Nodes ({len(nodes)}):')
    for node, coords in nodes.items():
        print(f'    - {node}: {coords}')
    print(f'  Edges ({len(edges)}):')
    for edge_key, edge_data in edges.items():
        print(f'    - {edge_key}: {edge_data}')

    # Show enumerated potential routes via recursion (for transparency)
    print('\n' + '='*60)
    print('[OPTIMIZE] Step 1: Route Enumeration (DFS Recursion)')
    print('='*60)
    paths = enumerate_paths(nodes, edges, start, end, max_depth=10)
    print(f'\nFound {len(paths)} possible paths')

    print('\n' + '='*60)
    print('[OPTIMIZE] Step 2: Build LP Model')
    print('='*60)
    model = LpProblem('Logistics_Optimization', LpMinimize)
    print('  → Created LpProblem instance')
    
    print('  → Creating binary decision variables for each edge')
    x = LpVariable.dicts('x', edges.keys(), cat='Binary')
    print(f'    Created {len(x)} binary variables: {list(x.keys())}')

    # Objective
    print('\n  → Setting objective function')
    if objective in ('time', 'fastest'):
        print(f'    Minimizing TIME')
        model += lpSum(x[e] * edges[e].get('time', 0) for e in edges), 'Minimize_Time'
    elif objective == 'distance':
        print(f'    Minimizing DISTANCE')
        model += lpSum(x[e] * edges[e].get('distance', 0) for e in edges), 'Minimize_Distance'
    else:
        print(f'    Minimizing COST (fuel)')
        model += lpSum(x[e] * edges[e].get('fuel', 0) for e in edges), 'Minimize_Cost'
    print('    ✓ Objective expression set')

    # Flow constraints
    print('\n  → Adding flow conservation constraints')
    for n_idx, n in enumerate(nodes):
        inflow = lpSum(x[(i, j)] for (i, j) in edges if j == n)
        outflow = lpSum(x[(i, j)] for (i, j) in edges if i == n)
        if n == start:
            model += outflow - inflow == 1, f'flow_start_{n}'
            print(f'    Constraint {n_idx+1}/{len(nodes)} (START/{n}): outflow - inflow = 1')
        elif n == end:
            model += inflow - outflow == 1, f'flow_end_{n}'
            print(f'    Constraint {n_idx+1}/{len(nodes)} (END/{n}): inflow - outflow = 1')
        else:
            model += inflow == outflow, f'flow_int_{n}'
            print(f'    Constraint {n_idx+1}/{len(nodes)} (INTERMEDIATE/{n}): inflow = outflow')

    if via:
        print(f'\n  → Adding VIA constraint(s) for node(s): {via}')
        if isinstance(via, list):
            for v in via:
                model += lpSum(x[(i, j)] for (i, j) in edges if j == v) == 1, f'via_{v}'
                print(f'    Node {v} must have exactly 1 incoming edge')
        else:
            model += lpSum(x[(i, j)] for (i, j) in edges if j == via) == 1, f'via_{via}'
            print(f'    Node {via} must have exactly 1 incoming edge')

    # Print LP formulation
    print('\n' + '='*60)
    print('[OPTIMIZE] Step 3: Write LP File')
    print('='*60)
    try:
        model.writeLP(lp_filename)
        print(f'  ✓ LP written to {lp_filename}')
        with open(lp_filename, 'r') as f:
            lp_text = f.read()
        print(f'  LP File size: {len(lp_text)} chars')
        print('\n  [LP FILE CONTENT - first 1500 chars]:')
        print('  ' + '\n  '.join(lp_text[:1500].split('\n')))
        if len(lp_text) > 1500:
            print(f'  ... ({len(lp_text)-1500} more chars)')
    except Exception as e:
        print(f'  ✗ Could not write/read LP file: {e}')

    print('\n' + '='*60)
    print('[OPTIMIZE] Step 4: Solve with CBC')
    print('='*60)
    solver = PULP_CBC_CMD(msg=1) if solver_msg else PULP_CBC_CMD(msg=0)
    result = model.solve(solver)
    print(f'  Solver result code: {result}')
    print(f'  Model status: {model.status} (1=optimal, 0=not solved, -1=infeasible, -2=unbounded)')
    if model.status != 1:
        print('  ⚠ Warning: optimal solution not found')
    print(f'  Objective value: {model.objective.value()}')

    print('\n' + '='*60)
    print('[OPTIMIZE] Step 5: Extract Solution')
    print('='*60)
    selected = []
    for e_idx, e in enumerate(edges):
        val = x[e].varValue
        print(f'  Variable {e_idx+1}/{len(edges)}: x{e} = {val}', end='')
        if val == 1:
            selected.append(edges[e])
            print(' ✓ SELECTED')
            print(f'    → Edge data: {edges[e]}')
        else:
            print(' ✗ not selected')
    
    print(f'\n  Total selected segments: {len(selected)}')
    return selected

print('✓ optimize defined')

# ============================================================================
# TEST 5: optimize with larger dummy graph (US Cities)
# ============================================================================
print('\n' + '#'*70)
print('# TEST 5: optimize Function with Full LP Formulation')
print('#'*70)

nodes_big = {
    'NYC': (40.7128, -74.0060),
    'Philadelphia': (39.9526, -75.1652),
    'Washington': (38.9072, -77.0369),
    'Atlanta': (33.7490, -84.3880)
}
edges_big = {
    ('NYC','Philadelphia'):{'from':'NYC','to':'Philadelphia','distance':95,'time':1.8,'fuel':47.5,'mode':'truck'},
    ('Philadelphia','Washington'):{'from':'Philadelphia','to':'Washington','distance':140,'time':2.5,'fuel':70.0,'mode':'truck'},
    ('Washington','Atlanta'):{'from':'Washington','to':'Atlanta','distance':640,'time':10.5,'fuel':320.0,'mode':'truck'},
    ('NYC','Washington'):{'from':'NYC','to':'Washington','distance':230,'time':3.8,'fuel':115.0,'mode':'truck'},
    ('Philadelphia','Atlanta'):{'from':'Philadelphia','to':'Atlanta','distance':780,'time':12.5,'fuel':390.0,'mode':'truck'}
}

print(f'\nInput: Find optimal route from NYC to Atlanta')
print(f'  Nodes: {list(nodes_big.keys())}')
print(f'  Edges: {len(edges_big)} possible routes')
print(f'  Objective: Minimize Cost')

selected = optimize(nodes_big, edges_big, 'NYC', 'Atlanta', objective='cost', lp_filename='test_model.lp', solver_msg=True)

print('\n✓ Test 5 complete - Selected route:')
summary_final = get_route_summary(selected)
print('\nFinal Summary:')
for key, val in summary_final.items():
    if key == 'fuel':
        print(f'  {key}: ${val:.2f}')
    elif key == 'time':
        print(f'  {key}: {val:.1f} hours')
    else:
        print(f'  {key}: {val} km')


✓ optimize defined with LP write and recursion enumeration


In [ ]:
def get_route_summary(route):
    print('\n' + '='*60)
    print('[GET_ROUTE_SUMMARY] Processing route')
    print('='*60)
    
    # Line: Check if route is empty
    if not route:
        print('  ⚠ Route is empty or None')
        return {}
    
    print(f'  Route has {len(route)} segments')
    
    # Line: Initialize totals
    totals = {'distance': 0, 'time': 0, 'fuel': 0}
    print('  → Initialized totals: distance=0, time=0, fuel=0')
    
    # Line: Iterate through each segment
    print('\n  Processing each segment:')
    for i, seg in enumerate(route, 1):
        print(f'\n  Segment {i}/{len(route)}:')
        print(f'    Data: {seg}')
        
        # Line: Extract distance
        dist = seg.get('distance', 0)
        print(f'    → distance: {dist} km')
        totals['distance'] += dist
        print(f'       Cumulative distance: {totals["distance"]} km')
        
        # Line: Extract time
        time_val = seg.get('time', 0)
        print(f'    → time: {time_val} hours')
        totals['time'] += time_val
        print(f'       Cumulative time: {totals["time"]} hours')
        
        # Line: Extract fuel
        fuel_val = seg.get('fuel', 0)
        print(f'    → fuel cost: ${fuel_val}')
        totals['fuel'] += fuel_val
        print(f'       Cumulative fuel cost: ${totals["fuel"]}')
    
    # Line: Return final totals
    print('\n  Final Totals:')
    print(f'    Distance: {totals["distance"]} km')
    print(f'    Time: {totals["time"]} hours')
    print(f'    Fuel Cost: ${totals["fuel"]:.2f}')
    return totals

print('✓ get_route_summary defined')

# ============================================================================
# TEST 2: get_route_summary with dummy route
# ============================================================================
print('\n' + '#'*70)
print('# TEST 2: get_route_summary Function')
print('#'*70)

dummy_route = [
    {'from': 'NYC', 'to': 'Philadelphia', 'distance': 95, 'time': 1.8, 'fuel': 47.5},
    {'from': 'Philadelphia', 'to': 'Washington', 'distance': 140, 'time': 2.5, 'fuel': 70.0}
]

print(f'\nInput: Route with {len(dummy_route)} segments')
for i, seg in enumerate(dummy_route, 1):
    print(f'  Segment {i}: {seg}')

summary = get_route_summary(dummy_route)

print(f'\n✓ Test 2 complete')
print(f'  Summary: {summary}')


✓ get_route_summary defined


In [ ]:
def prepare_vehicles_df(vehicles_df):
    print('\n' + '='*60)
    print('[PREPARE_VEHICLES_DF] Processing vehicle data')
    print('='*60)
    
    # Line: Show input
    print(f'\n  Input DataFrame ({len(vehicles_df)} rows):')
    print(vehicles_df.to_string())
    
    # Line: Create copy
    print('\n  → Creating copy of input dataframe')
    df = vehicles_df.copy()
    print('     ✓ Copy created')
    
    # Line: Process warehouse names to base_city
    print('\n  → Converting WarehouseName to base_city (lowercase, stripped)')
    df['base_city'] = df['WarehouseName'].str.lower().str.strip()
    for i, city in enumerate(df['base_city'], 1):
        print(f'     Row {i}: "{df.iloc[i-1]["WarehouseName"]}" → "{city}"')
    
    # Line: Process vehicle types
    print('\n  → Converting VehicleType to vehicle_type (lowercase, stripped)')
    df['vehicle_type'] = df['VehicleType'].str.lower().str.strip()
    for i, vtype in enumerate(df['vehicle_type'], 1):
        print(f'     Row {i}: "{df.iloc[i-1]["VehicleType"]}" → "{vtype}"')
    
    # Line: Convert capacity to numeric
    print('\n  → Converting VehicleCapacity to capacity_kg (numeric)')
    df['capacity_kg'] = pd.to_numeric(df['VehicleCapacity'], errors='coerce')
    for i, cap in enumerate(df['capacity_kg'], 1):
        print(f'     Row {i}: {cap} kg')
    
    # Line: Process departure times
    print('\n  → Extracting DepartureTime')
    df['departure_time'] = df['DepartureTime'].str.strip()
    for i, dept in enumerate(df['departure_time'], 1):
        print(f'     Row {i}: {dept}')

    # Line: Group by city and count
    print('\n  → Assigning vehicle numbers by city (vehicle_id_num)')
    df['vehicle_id_num'] = df.groupby('base_city').cumcount() + 1
    print('     Groupby city counts:')
    for i, row in df.iterrows():
        print(f'     Row {i+1}: city={row["base_city"]}, vehicle_id_num={row["vehicle_id_num"]}')
    
    # Line: Format vehicle ID
    print('\n  → Formatting vehicle_id: CITY_CODE-TYPE_CODE-NUMBER')
    def format_vid(row):
        city_code = row['base_city'][:3].upper()
        type_code = row['vehicle_type'][:3].upper()
        vid = f"{city_code}-{type_code}-{row['vehicle_id_num']:02d}"
        return vid
    df['vehicle_id'] = df.apply(format_vid, axis=1)
    for i, row in df.iterrows():
        print(f'     Row {i+1}: {row["vehicle_id"]} (from {row["base_city"]} {row["vehicle_type"]})')
    
    # Line: Map speed from VEHICLE_SPECS
    print('\n  → Mapping speed_kmph from VEHICLE_SPECS')
    df['speed_kmph'] = df['vehicle_type'].map(lambda x: VEHICLE_SPECS.get(x, {}).get('speed', 60))
    for i, row in df.iterrows():
        spec = VEHICLE_SPECS.get(row['vehicle_type'], {})
        print(f'     Row {i+1}: {row["vehicle_type"]} → {row["speed_kmph"]} kmph (spec: {spec})')
    
    # Line: Map cost from VEHICLE_SPECS
    print('\n  → Mapping cost_per_km from VEHICLE_SPECS')
    df['cost_per_km'] = df['vehicle_type'].map(lambda x: VEHICLE_SPECS.get(x, {}).get('cost', 0.5))
    for i, row in df.iterrows():
        spec = VEHICLE_SPECS.get(row['vehicle_type'], {})
        print(f'     Row {i+1}: {row["vehicle_type"]} → ${row["cost_per_km"]}/km (spec: {spec})')
    
    # Line: Select relevant columns
    print('\n  → Selecting relevant columns for output')
    output_cols = ['vehicle_id','base_city','vehicle_type','capacity_kg','speed_kmph','cost_per_km','departure_time']
    print(f'     Columns: {output_cols}')
    
    print('\n  Output DataFrame:')
    result = df[output_cols]
    print(result.to_string())
    
    return result

print('✓ prepare_vehicles_df defined')

# ============================================================================
# TEST 3: prepare_vehicles_df with dummy vehicle data
# ============================================================================
print('\n' + '#'*70)
print('# TEST 3: prepare_vehicles_df Function')
print('#'*70)

vehicles_raw = pd.DataFrame({
    'WarehouseName': ['New York', 'New York', 'Philadelphia', 'Philadelphia'],
    'VehicleType': ['Truck', 'Van', 'Truck', 'Bike'],
    'VehicleCapacity': [5000, 2000, 5000, 100],
    'DepartureTime': ['08:00', '09:00', '07:00', '10:30']
})

print(f'\nInput: {len(vehicles_raw)} vehicles from 2 cities')
print('Raw DataFrame:')
print(vehicles_raw)

vehicles_prepared = prepare_vehicles_df(vehicles_raw)

print(f'\n✓ Test 3 complete - Prepared {len(vehicles_prepared)} vehicles')


✓ prepare_vehicles_df defined


In [ ]:
def assign_vehicles_for_leg(vehicles_df, leg, total_goods, objective='cost'):
    print('\n' + '='*60)
    print('[ASSIGN_VEHICLES_FOR_LEG] Vehicle Assignment for Leg')
    print('='*60)
    
    # Line: Show input parameters
    print('\n  Input Parameters:')
    print(f'    leg: {leg}')
    print(f'    total_goods: {total_goods} kg')
    print(f'    objective: {objective} (time or cost)')
    
    # Line: Extract source city
    print('\n  → Extracting source city from leg["from"]')
    src = leg['from'].lower()
    print(f'    Source city: {src}')
    
    # Line: Filter available vehicles
    print('\n  → Filtering vehicles available at source city')
    available = vehicles_df[vehicles_df['base_city'].str.lower()==src].copy()
    print(f'    Found {len(available)} available vehicles:')
    if len(available) > 0:
        print(available.to_string())
    else:
        print('    ⚠ No vehicles available')
    
    # Line: Check if any vehicles available
    if available.empty:
        print('\n  ✗ RESULT: No vehicles available at this location')
        return None
    
    # Line: Create LP model
    print('\n' + '-'*60)
    print('  LP Model Setup:')
    print('-'*60)
    model = LpProblem('Vehicle_Assignment', LpMinimize)
    print('  → Created LP model: Vehicle_Assignment')
    
    # Line: Create decision variables
    print('  → Creating binary decision variables for each vehicle')
    x = LpVariable.dicts('v', available.index, cat='Binary')
    print(f'    Variables: {list(x.keys())}')
    
    # Line: Set objective
    print('\n  → Setting objective function')
    if objective=='time':
        print(f'    Minimizing travel TIME')
        model += lpSum(x[i] * (leg['distance']/available.loc[i,'speed_kmph']) for i in available.index)
        for i in available.index:
            speed = available.loc[i,'speed_kmph']
            travel_time = leg['distance']/speed
            print(f'      Variable v[{i}]: {leg["distance"]} km / {speed} kmph = {travel_time:.2f} hours')
    else:
        print(f'    Minimizing travel COST')
        model += lpSum(x[i] * available.loc[i,'cost_per_km'] * leg['distance'] for i in available.index)
        for i in available.index:
            cost_km = available.loc[i,'cost_per_km']
            total_cost = cost_km * leg['distance']
            print(f'      Variable v[{i}]: ${cost_km}/km × {leg["distance"]} km = ${total_cost:.2f}')
    
    # Line: Add capacity constraint
    print(f'\n  → Adding capacity constraint: sum(capacity × selected) >= {total_goods} kg')
    capacity_expr = lpSum(x[i]*available.loc[i,'capacity_kg'] for i in available.index)
    model += capacity_expr >= total_goods
    print(f'    Available capacities:')
    for i in available.index:
        cap = available.loc[i,'capacity_kg']
        vid = available.loc[i,'vehicle_id']
        print(f'      v[{i}] ({vid}): {cap} kg')
    print(f'    Constraint: capacity_sum >= {total_goods} kg')
    
    # Line: Write LP file
    print('\n  → Writing LP file: assign_model.lp')
    model.writeLP('assign_model.lp')
    print('    ✓ File written')
    try:
        lp_content = open('assign_model.lp').read()
        print(f'    LP file size: {len(lp_content)} chars')
        print('    [First 800 chars]:')
        print('    ' + '\n    '.join(lp_content[:800].split('\n')))
    except Exception as e:
        print(f'    Could not read file: {e}')
    
    # Line: Solve
    print('\n' + '-'*60)
    print('  Solving:')
    print('-'*60)
    res = model.solve(PULP_CBC_CMD(msg=1))
    print(f'  Solve result code: {res}')
    print(f'  Model status: {model.status}')
    
    # Line: Extract solution
    print('\n' + '-'*60)
    print('  Solution Extraction:')
    print('-'*60)
    assigned = []
    remain = total_goods
    print(f'  Remaining goods to assign: {remain} kg')
    
    for i in available.index:
        var_val = x[i].varValue
        vid = available.loc[i,'vehicle_id']
        cap = available.loc[i,'capacity_kg']
        cost_per_km = available.loc[i,'cost_per_km']
        speed = available.loc[i,'speed_kmph']
        
        print(f'\n  Vehicle {i}: {vid}')
        print(f'    Variable value (selected): {var_val}')
        print(f'    Capacity: {cap} kg')
        print(f'    Remaining to assign: {remain} kg')
        
        if var_val==1 and remain>0:
            load = min(cap, remain)
            depart = available.loc[i,'departure_time'] if pd.notna(available.loc[i,'departure_time']) else '08:00'
            travel_time = leg['distance']/speed
            travel_cost = cost_per_km * leg['distance']
            
            print(f'    ✓ ASSIGNED!')
            print(f'      Load: {load} kg (capacity: {cap} kg)')
            print(f'      Departure: {depart}')
            print(f'      Travel time: {leg["distance"]}km / {speed}kmph = {travel_time:.2f} hours')
            print(f'      Travel cost: ${travel_cost:.2f}')
            
            assigned.append({
                'vehicle_id':vid,
                'load_kg':load,
                'departure':depart,
                'travel_time':travel_time
            })
            remain -= load
            print(f'      Remaining after: {remain} kg')
        else:
            if var_val != 1:
                print(f'    ✗ NOT selected by optimizer')
            else:
                print(f'    ✗ Already assigned to other legs')
    
    # Line: Create result
    print('\n' + '='*60)
    print('  FINAL ASSIGNMENT RESULT:')
    print('='*60)
    result = {'from':leg['from'],'to':leg['to'],'vehicles':assigned}
    print(f'  From: {result["from"]}')
    print(f'  To: {result["to"]}')
    print(f'  Assigned vehicles ({len(assigned)}):')
    for v in assigned:
        print(f'    - {v}')
    print(f'  Remaining unassigned: {remain} kg')
    
    return result

print('✓ assign_vehicles_for_leg defined')

# ============================================================================
# TEST 4: assign_vehicles_for_leg with prepared vehicles
# ============================================================================
print('\n' + '#'*70)
print('# TEST 4: assign_vehicles_for_leg Function')
print('#'*70)

leg_1 = {'from': 'New York', 'to': 'Philadelphia', 'distance': 95, 'time': 1.8}
total_goods_1 = 4500  # kg

print(f'\nInput: Assigning vehicles for leg NY → Philadelphia')
print(f'  Distance: {leg_1["distance"]} km')
print(f'  Goods to transport: {total_goods_1} kg')
print(f'  Objective: cost')

assignment_1 = assign_vehicles_for_leg(vehicles_prepared, leg_1, total_goods_1, objective='cost')
print(f'\n✓ Test 4 complete - Assignment result:')
print(assignment_1)


✓ assign_vehicles_for_leg defined
